In [1]:
import os
from pathlib import Path

p = Path.cwd()
os.chdir(p.parent)
os.getcwd()

'/root/repos/lightning/HousePricing'

In [2]:
from dotenv import dotenv_values
import os

envs = ["secret.env", "test.env"]

for fenv in envs:
    file = os.path.join("env", fenv)
    config = dotenv_values(file)  # load sensitive variables
    print(config.keys())
    for c, v in config.items():
        os.environ[c] = v

odict_keys(['WANDB_API_KEY', 'WANDB_NAME'])
odict_keys(['WANDB_NOTEBOOK_NAME'])


In [3]:
import wandb
import os

wandb_key = os.environ["WANDB_API_KEY"]
wandb.login(key=wandb_key)

wandb: Currently logged in as: wilber-quito (deepsat). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [4]:
# Preparing data to be used
from src.data import HousePricingDataModule

data_module = HousePricingDataModule()
data_module.prepare_data()

[INFO]: Skipping downloading data. Data is already downloaded
[INFO]: Reloading set up data...


In [5]:
import pandas as pd

artifacts = pd.read_csv("artifacts.csv")
artifacts

,run_name,best_model,fit_accelerator,fit_max_epochs,fit_patience,fit_lr,fit_batch_size,fit_in_features,fit_validation_size
0,Mr._ZI1EUZUC,./lightning_logs/mrsat7bm/checkpoints/house_pr...,gpu,1000,100,0.001,256,244,0.1
1,Gerald_4LIP1BLK,./lightning_logs/3478itnz/checkpoints/house_pr...,gpu,1000,100,0.001,256,244,0.1
2,Stacey_Y41HGQZW,./lightning_logs/27yrkxwa/checkpoints/house_pr...,gpu,1000,100,0.001,256,244,0.1


In [6]:
artifacts = artifacts.to_dict(orient="records")
artifacts

[{'run_name': 'Mr._ZI1EUZUC',
  'best_model': './lightning_logs/mrsat7bm/checkpoints/house_pricing-epoch=956-val_loss=383568800.00.ckpt',
  'fit_accelerator': 'gpu',
  'fit_max_epochs': 1000,
  'fit_patience': 100,
  'fit_lr': 0.001,
  'fit_batch_size': 256,
  'fit_in_features': 244,
  'fit_validation_size': 0.1},
 {'run_name': 'Gerald_4LIP1BLK',
  'best_model': './lightning_logs/3478itnz/checkpoints/house_pricing-epoch=831-val_loss=389454272.00.ckpt',
  'fit_accelerator': 'gpu',
  'fit_max_epochs': 1000,
  'fit_patience': 100,
  'fit_lr': 0.001,
  'fit_batch_size': 256,
  'fit_in_features': 244,
  'fit_validation_size': 0.1},
 {'run_name': 'Stacey_Y41HGQZW',
  'best_model': './lightning_logs/27yrkxwa/checkpoints/house_pricing-epoch=818-val_loss=392531328.00.ckpt',
  'fit_accelerator': 'gpu',
  'fit_max_epochs': 1000,
  'fit_patience': 100,
  'fit_lr': 0.001,
  'fit_batch_size': 256,
  'fit_in_features': 244,
  'fit_validation_size': 0.1}]

In [7]:
import os
from lightning import Trainer
import torch

from pytorch_lightning.loggers import WandbLogger

from src.model import NeuralNetwork

project_name = os.environ["WANDB_NAME"]

accelerator = "gpu" if torch.cuda.is_available() else "cpu"
in_features = data_module.data_features()

config = {
    "accelerator": accelerator,
    "in_features": in_features,
}

test_results = list()

for artifact in artifacts:

    run_name = artifact["run_name"]
    best_model = artifact["best_model"]

    wandb.init(
        job_type="test",
        name=run_name,
        project=project_name,
        config=config,
    )

    model = NeuralNetwork.load_from_checkpoint(
        checkpoint_path=best_model, input_size=in_features
    )

    logger = WandbLogger(checkpoint_name=run_name)

    trainer = Trainer(
        accelerator=wandb.config["accelerator"], logger=logger, log_every_n_steps=1
    )

    test_loss = trainer.test(model, datamodule=data_module)

    data = {"run_name": run_name, "best_model": best_model, "test_loss": test_loss}

    test_results.append(data)

    wandb.finish()

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


[INFO]: Input size: 244


/root/miniconda3/envs/pricehousing/lib/python3.9/site-packages/pytorch_lightning/loggers/wandb.py:391: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/root/miniconda3/envs/pricehousing/lib/python3.9/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


[INFO]: Skipping downloading data. Data is already downloaded
[INFO]: Reloading set up data...
[INFO]: Setting up test dataset/s
[INFO]: Test dataloader size: 2
Testing DataLoader 0: 100%|██████████| 2/2 [00:00<00:00, 21.34it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_loss               633113280.0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


epoch,▁
test_loss,▁
trainer/global_step,▁
epoch,0
test_loss,633113280.0
trainer/global_step,0


...

[INFO]: Input size: 244
[INFO]: Skipping downloading data. Data is already downloaded
[INFO]: Reloading set up data...
[INFO]: Setting up test dataset/s
[INFO]: Test dataloader size: 2
Testing DataLoader 0: 100%|██████████| 2/2 [00:00<00:00, 69.72it/s] 
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_loss               654472576.0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


epoch,▁
test_loss,▁
trainer/global_step,▁
epoch,0
test_loss,654472576.0
trainer/global_step,0


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/root/miniconda3/envs/pricehousing/lib/python3.9/site-packages/pytorch_lightning/loggers/wandb.py:391: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/root/miniconda3/envs/pricehousing/lib/python3.9/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


[INFO]: Input size: 244
[INFO]: Skipping downloading data. Data is already downloaded
[INFO]: Reloading set up data...
[INFO]: Setting up test dataset/s
[INFO]: Test dataloader size: 2
Testing DataLoader 0: 100%|██████████| 2/2 [00:00<00:00, 147.96it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_loss               642005888.0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


epoch,▁
test_loss,▁
trainer/global_step,▁
epoch,0
test_loss,642005888.0
trainer/global_step,0


In [8]:
test_results

[{'run_name': 'Mr._ZI1EUZUC',
  'best_model': './lightning_logs/mrsat7bm/checkpoints/house_pricing-epoch=956-val_loss=383568800.00.ckpt',
  'test_loss': [{'test_loss': 633113280.0}]},
 {'run_name': 'Gerald_4LIP1BLK',
  'best_model': './lightning_logs/3478itnz/checkpoints/house_pricing-epoch=831-val_loss=389454272.00.ckpt',
  'test_loss': [{'test_loss': 654472576.0}]},
 {'run_name': 'Stacey_Y41HGQZW',
  'best_model': './lightning_logs/27yrkxwa/checkpoints/house_pricing-epoch=818-val_loss=392531328.00.ckpt',
  'test_loss': [{'test_loss': 642005888.0}]}]